In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('appartments.csv').drop(22) # this rows has values as column name because of web scrapped data 
df.head()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Bowling Alley', 'Mini Theatre', 'Manicured G..."
2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63,...","['AIPL Business Club Sector 62', 'Heritage Xpe...","{'AIPL Business Club Sector 62': '2.7 Km', 'He...",https://www.99acres.com/adani-brahma-samsara-v...,{'3 BHK': {'building_type': 'Independent Floor...,"['Terrace Garden', 'Gazebo', 'Fountain', 'Amph..."
3,Sobha City,"2, 3, 4 BHK Apartment in Sector 108, Gurgaon","['The Shikshiyan School', 'WTC Plaza', 'Luxus ...","{'The Shikshiyan School': '2.9 KM', 'WTC Plaza...",https://www.99acres.com/sobha-city-sector-108-...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Volley Ball Court', 'Aerobi..."
4,Signature Global City 93,"2, 3 BHK Independent Floor in Sector 93 Gurgaon","['Pranavananda Int. School', 'DLF Site central...","{'Pranavananda Int. School': '450 m', 'DLF Sit...",https://www.99acres.com/signature-global-city-...,{'2 BHK': {'building_type': 'Independent Floor...,"['Mini Theatre', 'Doctor on Call', 'Concierge ..."


In [3]:
df.iloc[2]

PropertyName                                Adani Brahma Samsara Vilasa
PropertySubName       Land, 3, 4 BHK Independent Floor in Sector 63,...
NearbyLocations       ['AIPL Business Club Sector 62', 'Heritage Xpe...
LocationAdvantages    {'AIPL Business Club Sector 62': '2.7 Km', 'He...
Link                  https://www.99acres.com/adani-brahma-samsara-v...
PriceDetails          {'3 BHK': {'building_type': 'Independent Floor...
TopFacilities         ['Terrace Garden', 'Gazebo', 'Fountain', 'Amph...
Name: 2, dtype: object

# studying which columns are good for making system

In [4]:
# property name we need
# propertysubname
df.iloc[2][1]

'Land, 3, 4 BHK Independent Floor in Sector 63, Gurgaon'

In [5]:
 df.iloc[3][1]

'2, 3, 4 BHK Apartment in Sector 108, Gurgaon'

In [6]:
df.iloc[4][1]

'2, 3 BHK Independent Floor in Sector 93 Gurgaon'

In [7]:
# nearbylocations
df.iloc[2][2]

"['AIPL Business Club Sector 62', 'Heritage Xperiential Learning School', 'CK Birla Hospital', 'Paras Trinity Mall Sector 63', 'Rapid Metro Station Sector 56']"

In [8]:
df.iloc[3][2]

"['The Shikshiyan School', 'WTC Plaza', 'Luxus Haritma Resort', 'BSF Golf Course', 'Dwarka Expressway']"

In [9]:
# locationAdvantages
df.iloc[2][3]

"{'AIPL Business Club Sector 62': '2.7 Km', 'Heritage Xperiential Learning School': '2 Km', 'CK Birla Hospital': '2.5 Km', 'Paras Trinity Mall Sector 63': '3.5 Km', 'Rapid Metro Station Sector 56': '3.8 Km', 'De Adventure Park': '6.8 Km', 'Golf Course Ext Rd': '99 Meter', 'DoubleTree by Hilton Hotel Gurgaon': '3.6 Km', 'KIIT College of Engineering Sohna Road': '8.4 Km', 'Mehrauli-Gurgaon Road': '11.8 Km', 'Indira Gandhi International Airport': '21.1 Km', 'Nirvana Rd': '160 Meter', 'TERI Golf Course': '8.7 Km'}"

In [10]:
df.iloc[3][3]

"{'The Shikshiyan School': '2.9 KM', 'WTC Plaza': '4 KM', 'Luxus Haritma Resort': '4.2 KM', 'BSF Golf Course': '4.7 KM', 'Dwarka Expressway': '5.1 Km', 'Rions Hospital': '5.6 Km', 'Gurgaon': '5.9 KM', 'The NorthCap University': '9.2 KM', 'Dwarka Sector 21': '9.9 KM', 'Nehru Stadium': '10.7  KM', 'Fun N Food WaterPark': '12.1 KM', 'IGI Airport': '19.3 Km', 'Vasant Kunj': '20.8 KM'}"

## nearbyLocations is a subset of locationAdvantages 
## drop nearbyLocations

In [11]:
df.iloc[2][4]

'https://www.99acres.com/adani-brahma-samsara-vilasa-sector-63-gurgaon-npxid-r327995'

In [12]:
df.iloc[2][5]

"{'3 BHK': {'building_type': 'Independent Floor', 'area_type': 'Super Built-up Area', 'area': '1,800 - 3,150 sq.ft.', 'price-range': '₹ 2.43 - 15.75 Cr'}, '4 BHK': {'building_type': 'Independent Floor', 'area_type': 'Super Built-up Area', 'area': '2,750 - 4,500 sq.ft.', 'price-range': '₹ 3.36 - 22.5 Cr'}, 'Land': {'building_type': '', 'area_type': 'Plot Area', 'area': '500 - 4,329 sq.ft.', 'price-range': '₹ 2.05 - 41.13 Cr'}}"

In [13]:
df.iloc[3][5]

"{'2 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '1,381 - 1,692 sq.ft.', 'price-range': '₹ 1.55 - 3.21 Cr'}, '3 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '1,711 - 2,343 sq.ft.', 'price-range': '₹ 1.76 - 4.79 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '2,423 - 2,963 sq.ft.', 'price-range': '₹ 2.5 - 6.06 Cr'}}"

## propertySubtypes is a subset of Pricedetails
## drop propertySubtypes

In [14]:
# TOPfacilites 
df.iloc[2][6]

"['Terrace Garden', 'Gazebo', 'Fountain', 'Amphitheatre', 'Party Lawn', 'Basketball Court', 'Badminton Court', 'Yoga/Meditation Area', 'Indoor Games']"

In [15]:
df.iloc[3][6]

"['Swimming Pool', 'Volley Ball Court', 'Aerobics Centre', 'Card Room', 'Barbecue', 'Sauna', 'Steam Room', 'Creche/Day care', 'Skating Rink']"

###  usage of 3 col. - topfacilites , pricedetails,locationadvantage

### strategy individual 3 recommedor of each col , and just combined result of all three will be output of top 5 available socities 

In [16]:
# top facilities

In [17]:
df[['PropertyName','TopFacilities']]['TopFacilities'][0]

"['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']"

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246 entries, 0 to 246
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PropertyName        246 non-null    object
 1   PropertySubName     246 non-null    object
 2   NearbyLocations     246 non-null    object
 3   LocationAdvantages  246 non-null    object
 4   Link                246 non-null    object
 5   PriceDetails        246 non-null    object
 6   TopFacilities       246 non-null    object
dtypes: object(7)
memory usage: 23.5+ KB


In [19]:
def extract_list(s):
    return re.findall(r"'(.*?)'", s)

df['TopFacilities'] = df['TopFacilities'].apply(extract_list)

In [20]:
df['TopFacilities']

0      [Swimming Pool, Salon, Restaurant, Spa, Cafete...
1      [Bowling Alley, Mini Theatre, Manicured Garden...
2      [Terrace Garden, Gazebo, Fountain, Amphitheatr...
3      [Swimming Pool, Volley Ball Court, Aerobics Ce...
4      [Mini Theatre, Doctor on Call, Concierge Servi...
                             ...                        
242    [Swimming Pool, Medical Centre, Laundry, Salon...
243    [Shopping Centre, Community Hall, 24x7 Securit...
244    [Bus Shelter, Swimming Pool, Business Lounge, ...
245    [Swimming Pool, Card Room, Piped Gas, Pool Tab...
246    [Swimming Pool, School, Solar Lighting, Aerobi...
Name: TopFacilities, Length: 246, dtype: object

In [21]:
df['FacilitiesStr'] = df['TopFacilities'].apply(' '.join)

In [22]:
df['FacilitiesStr']

0      Swimming Pool Salon Restaurant Spa Cafeteria S...
1      Bowling Alley Mini Theatre Manicured Garden Sw...
2      Terrace Garden Gazebo Fountain Amphitheatre Pa...
3      Swimming Pool Volley Ball Court Aerobics Centr...
4      Mini Theatre Doctor on Call Concierge Service ...
                             ...                        
242    Swimming Pool Medical Centre Laundry Salon Gro...
243    Shopping Centre Community Hall 24x7 Security G...
244    Bus Shelter Swimming Pool Business Lounge Read...
245    Swimming Pool Card Room Piped Gas Pool Table S...
246    Swimming Pool School Solar Lighting Aerobics C...
Name: FacilitiesStr, Length: 246, dtype: object

In [23]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1,2))

In [24]:
tfidf_matrix= tfidf_vectorizer.fit_transform(df['FacilitiesStr'])

In [25]:
tfidf_matrix[0]

<1x953 sparse matrix of type '<class 'numpy.float64'>'
	with 27 stored elements in Compressed Sparse Row format>

In [26]:
tfidf_matrix.toarray()[0] # numerical representation of--->  Swimming Pool Salon Restaurant Spa Cafeteria S..

array([0.        , 0.        , 0.        , 0.18809342, 0.18809342,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [27]:
tfidf_matrix.shape

(246, 953)

In [28]:
cosine_sim1 = cosine_similarity(tfidf_matrix,tfidf_matrix)

In [29]:
p = 'DLF The Arbour'

idx = df.index[ df['PropertyName']==p ].tolist()[0] # index fetch of particular property
idx

11

In [30]:
cosine_sim1[idx] #get the pairwise similarity scores with that property

array([0.38885046, 0.03223439, 0.11672504, 0.01006389, 0.02247625,
       0.17940741, 0.02514308, 0.20622569, 0.02251413, 0.03417478,
       0.03567897, 1.        , 0.02577425, 0.02052461, 0.34512624,
       0.03749273, 0.11470799, 0.07544187, 0.35741312, 0.01308776,
       0.01136025, 0.0246845 , 0.01228715, 0.02174817, 0.01089504,
       0.02316634, 0.01425171, 0.00955051, 0.02333949, 0.31500236,
       0.02201931, 0.21466634, 0.17813746, 0.02185594, 0.01132816,
       0.02245775, 0.32025139, 0.03003836, 0.01136488, 0.0100986 ,
       0.01241946, 0.02575203, 0.01070536, 0.02402016, 0.09759408,
       0.17709522, 0.01213688, 0.00914781, 0.0106276 , 0.09585274,
       0.01822294, 0.01057304, 0.0108057 , 0.28077108, 0.04140942,
       0.02292972, 0.01182373, 0.01526512, 0.03956037, 0.01438962,
       0.01687644, 0.02222693, 0.37679772, 0.45293821, 0.32498474,
       0.01288932, 0.23352887, 0.02431145, 0.2313416 , 0.01505017,
       0.02228228, 0.02423885, 0.3882162 , 0.03006416, 0.02362

In [31]:
sim_scores = list(enumerate(cosine_sim1[idx])) # dlf abour ka similarity score baaki property se
sim_scores

[(0, 0.38885046199432893),
 (1, 0.03223439184656369),
 (2, 0.11672503712096931),
 (3, 0.010063889472689515),
 (4, 0.02247625396597818),
 (5, 0.1794074075441662),
 (6, 0.025143076472187182),
 (7, 0.20622568609389147),
 (8, 0.022514133739354406),
 (9, 0.03417478413059936),
 (10, 0.03567897045421953),
 (11, 1.0000000000000002),
 (12, 0.02577424531172577),
 (13, 0.020524613605297677),
 (14, 0.3451262362291603),
 (15, 0.037492726442249054),
 (16, 0.11470799144443575),
 (17, 0.07544186924621539),
 (18, 0.35741312385828006),
 (19, 0.013087760472195235),
 (20, 0.011360247925424062),
 (21, 0.02468450149581614),
 (22, 0.012287153821765226),
 (23, 0.02174817104318221),
 (24, 0.010895043067047583),
 (25, 0.02316633572395576),
 (26, 0.01425171224278603),
 (27, 0.00955050782145329),
 (28, 0.023339486792302723),
 (29, 0.315002361770242),
 (30, 0.02201930872554653),
 (31, 0.2146663412969806),
 (32, 0.17813745693789748),
 (33, 0.021855939925838334),
 (34, 0.011328164015043404),
 (35, 0.0224577457902342

In [32]:
# Sort the properties based on the similarity scores
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores

[(11, 1.0000000000000002),
 (63, 0.4529382062441955),
 (216, 0.4199606322926784),
 (92, 0.4166584649363288),
 (153, 0.398954234680194),
 (0, 0.38885046199432893),
 (72, 0.38821619878613484),
 (62, 0.3767977219141552),
 (18, 0.35741312385828006),
 (185, 0.35334766728957717),
 (242, 0.34844371350826064),
 (14, 0.3451262362291603),
 (134, 0.32763578071449695),
 (64, 0.32498473830403196),
 (36, 0.3202513860830745),
 (29, 0.315002361770242),
 (236, 0.2987869362705078),
 (168, 0.2897800851644227),
 (53, 0.2807710756996148),
 (108, 0.2796849177261259),
 (198, 0.2722354421798556),
 (203, 0.2650747212649878),
 (204, 0.25559851542592754),
 (218, 0.2526985816730312),
 (191, 0.24949635030536815),
 (221, 0.24008496524362596),
 (240, 0.23867044693345305),
 (66, 0.23352886738575768),
 (160, 0.23179838478665726),
 (68, 0.23134159558658823),
 (115, 0.22147879043546226),
 (31, 0.2146663412969806),
 (7, 0.20622568609389147),
 (210, 0.20075618320015828),
 (94, 0.18791602627666765),
 (138, 0.18657126247963

In [33]:
# get sim_scores top 5 
sim_scores[1:6]

[(63, 0.4529382062441955),
 (216, 0.4199606322926784),
 (92, 0.4166584649363288),
 (153, 0.398954234680194),
 (0, 0.38885046199432893)]

In [34]:
def recommend_properties(property_name, cosine_sim = cosine_sim1):
    
    idx = df.index[ df['PropertyName'] == property_name ].tolist()[0]
    
    sim_scores = list( enumerate( cosine_sim1[idx] ) )

    sim_scores = sorted( sim_scores , key = lambda x:x[1] , reverse = True)
    
    sim_scores = sim_scores[1:6]
    
    property_indices = [i[0] for i in sim_scores ]
    
    recommendation_df = pd.DataFrame({
        'PropertyName':df['PropertyName'].iloc[property_indices],
        'SimilarityScore': sim_scores
    })
    
    return recommendation_df

In [35]:
recommend_properties('DLF The Arbour')

,PropertyName,SimilarityScore
64,Ace Palm Floors,"(63, 0.4529382062441955)"
217,Yashika 104,"(216, 0.4199606322926784)"
93,JMS The Nation,"(92, 0.4166584649363288)"
154,India Rashtra,"(153, 0.398954234680194)"
0,Smartworld One DXP,"(0, 0.38885046199432893)"


In [36]:
# toughest column - pricedetails
df.iloc[2][5]

"{'3 BHK': {'building_type': 'Independent Floor', 'area_type': 'Super Built-up Area', 'area': '1,800 - 3,150 sq.ft.', 'price-range': '₹ 2.43 - 15.75 Cr'}, '4 BHK': {'building_type': 'Independent Floor', 'area_type': 'Super Built-up Area', 'area': '2,750 - 4,500 sq.ft.', 'price-range': '₹ 3.36 - 22.5 Cr'}, 'Land': {'building_type': '', 'area_type': 'Plot Area', 'area': '500 - 4,329 sq.ft.', 'price-range': '₹ 2.05 - 41.13 Cr'}}"

In [37]:
df.iloc[22] # observed in ms excel while just scrolling the data , so we need to remove it 

PropertyName                                                M3M Capital
PropertySubName            2, 3, 4 BHK Apartment in Sector 113, Gurgaon
NearbyLocations       ['Dwarka Expy', 'Shivani public school', 'Bagh...
LocationAdvantages    {'Dwarka Expy': '10 m', 'Shivani public school...
Link                  https://www.99acres.com/m3m-capital-sector-113...
PriceDetails          {'2 BHK': {'building_type': 'Apartment', 'area...
TopFacilities         [Swimming Pool, Lounge, Bar/Chill-Out Lounge, ...
FacilitiesStr         Swimming Pool Lounge Bar/Chill-Out Lounge ATM ...
Name: 23, dtype: object

In [38]:
# used claudai to generate this code 
import pandas as pd
import json

# Load the dataset
df_appartments = pd.read_csv('appartments.csv').drop(22)

# Function to parse and extract the required features from the PriceDetails column
def refined_parse_modified_v2(detail_str):
    try:
        details = json.loads(detail_str.replace("'", "\""))
    except:
        return {}

    extracted = {}
    for bhk, detail in details.items():
        # Extract building type
        extracted[f'building type_{bhk}'] = detail.get('building_type')

        # Parsing area details
        area = detail.get('area', '')
        area_parts = area.split('-')
        if len(area_parts) == 1:
            try:
                value = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area low {bhk}'] = value
                extracted[f'area high {bhk}'] = value
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None
        elif len(area_parts) == 2:
            try:
                extracted[f'area low {bhk}'] = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area high {bhk}'] = float(area_parts[1].replace(',', '').replace(' sq.ft.', '').strip())
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None

        # Parsing price details
        price_range = detail.get('price-range', '')
        price_parts = price_range.split('-')
        if len(price_parts) == 2:
            try:
                extracted[f'price low {bhk}'] = float(price_parts[0].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                extracted[f'price high {bhk}'] = float(price_parts[1].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                if 'L' in price_parts[0]:
                    extracted[f'price low {bhk}'] /= 100
                if 'L' in price_parts[1]:
                    extracted[f'price high {bhk}'] /= 100
            except:
                extracted[f'price low {bhk}'] = None
                extracted[f'price high {bhk}'] = None

    return extracted
# Apply the refined parsing and generate the new DataFrame structure
data_refined = []

for _, row in df_appartments.iterrows():
    features = refined_parse_modified_v2(row['PriceDetails'])
    
    # Construct a new row for the transformed dataframe
    new_row = {'PropertyName': row['PropertyName']}
    
    # Populate the new row with extracted features
    for config in ['1 BHK', '2 BHK', '3 BHK', '4 BHK', '5 BHK', '6 BHK', '1 RK', 'Land']:
        new_row[f'building type_{config}'] = features.get(f'building type_{config}')
        new_row[f'area low {config}'] = features.get(f'area low {config}')
        new_row[f'area high {config}'] = features.get(f'area high {config}')
        new_row[f'price low {config}'] = features.get(f'price low {config}')
        new_row[f'price high {config}'] = features.get(f'price high {config}')
    
    data_refined.append(new_row)

df_final_refined_v2 = pd.DataFrame(data_refined).set_index('PropertyName')

In [39]:
df_final_refined_v2['building type_Land'] = df_final_refined_v2['building type_Land'].replace({'':'Land'})

In [40]:
df_final_refined_v2

,building type_1 BHK,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,building type_2 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,...,building type_1 RK,area low 1 RK,area high 1 RK,price low 1 RK,price high 1 RK,building type_Land,area low Land,area high Land,price low Land,price high Land
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,None,NaN,NaN,NaN,NaN,Apartment,1370.0,1370.0,2.0000,2.40,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
M3M Crown,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Adani Brahma Samsara Vilasa,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,Land,500.0,4329.0,2.05,41.13
Sobha City,None,NaN,NaN,NaN,NaN,Apartment,1381.0,1692.0,1.5500,3.21,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Signature Global City 93,None,NaN,NaN,NaN,NaN,Independent Floor,981.0,1118.0,0.9301,1.06,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,None,NaN,NaN,NaN,NaN,Apartment,964.0,964.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Pyramid Urban Homes 2,Apartment,335.0,398.0,23.45,0.2786,Apartment,500.0,625.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
Satya The Hermitage,None,NaN,NaN,NaN,NaN,Apartment,1450.0,1450.0,NaN,NaN,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [41]:
df_final_refined_v2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246 entries, Smartworld One DXP to SS The Coralwood
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   building type_1 BHK  18 non-null     object 
 1   area low 1 BHK       18 non-null     float64
 2   area high 1 BHK      18 non-null     float64
 3   price low 1 BHK      5 non-null      float64
 4   price high 1 BHK     5 non-null      float64
 5   building type_2 BHK  121 non-null    object 
 6   area low 2 BHK       121 non-null    float64
 7   area high 2 BHK      121 non-null    float64
 8   price low 2 BHK      72 non-null     float64
 9   price high 2 BHK     72 non-null     float64
 10  building type_3 BHK  186 non-null    object 
 11  area low 3 BHK       186 non-null    float64
 12  area high 3 BHK      186 non-null    float64
 13  price low 3 BHK      133 non-null    float64
 14  price high 3 BHK     133 non-null    float64
 15  building type_4

In [42]:
df_final_refined_v2.select_dtypes(include=['object'])

,building type_1 BHK,building type_2 BHK,building type_3 BHK,building type_4 BHK,building type_5 BHK,building type_6 BHK,building type_1 RK,building type_Land
PropertyName,,,,,,,,
Smartworld One DXP,None,Apartment,Apartment,Apartment,None,None,None,None
M3M Crown,None,None,Apartment,Apartment,None,None,None,None
Adani Brahma Samsara Vilasa,None,None,Independent Floor,Independent Floor,None,None,None,Land
Sobha City,None,Apartment,Apartment,Apartment,None,None,None,None
Signature Global City 93,None,Independent Floor,Independent Floor,None,None,None,None,None
...,...,...,...,...,...,...,...,...
DLF Princeton Estate,None,Apartment,Apartment,Apartment,None,None,None,None
Pyramid Urban Homes 2,Apartment,Apartment,None,None,None,None,None,None
Satya The Hermitage,None,Apartment,Apartment,Apartment,Apartment,None,None,None


In [43]:
df_final_refined_v2.select_dtypes(include = ['object'] ).columns

Index(['building type_1 BHK', 'building type_2 BHK', 'building type_3 BHK',
       'building type_4 BHK', 'building type_5 BHK', 'building type_6 BHK',
       'building type_1 RK', 'building type_Land'],
      dtype='object')

In [44]:
categorical_cols = df_final_refined_v2.select_dtypes(include = ['object'] ).columns.tolist()
categorical_cols

['building type_1 BHK',
 'building type_2 BHK',
 'building type_3 BHK',
 'building type_4 BHK',
 'building type_5 BHK',
 'building type_6 BHK',
 'building type_1 RK',
 'building type_Land']

In [45]:
ohe_df = pd.get_dummies(df_final_refined_v2, columns = categorical_cols, drop_first=True)
ohe_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246 entries, Smartworld One DXP to SS The Coralwood
Data columns (total 43 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   area low 1 BHK                         18 non-null     float64
 1   area high 1 BHK                        18 non-null     float64
 2   price low 1 BHK                        5 non-null      float64
 3   price high 1 BHK                       5 non-null      float64
 4   area low 2 BHK                         121 non-null    float64
 5   area high 2 BHK                        121 non-null    float64
 6   price low 2 BHK                        72 non-null     float64
 7   price high 2 BHK                       72 non-null     float64
 8   area low 3 BHK                         186 non-null    float64
 9   area high 3 BHK                        186 non-null    float64
 10  price low 3 BHK                        133 non-nu

In [46]:
ohe_df

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,...,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,NaN,NaN,NaN,NaN,1370.0,1370.0,2.0000,2.40,1850.0,2050.0,...,False,False,False,False,False,False,False,False,False,False
M3M Crown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1605.0,2170.0,...,False,False,False,False,False,False,False,False,False,False
Adani Brahma Samsara Vilasa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1800.0,3150.0,...,False,False,True,False,False,True,False,False,False,False
Sobha City,NaN,NaN,NaN,NaN,1381.0,1692.0,1.5500,3.21,1711.0,2343.0,...,False,False,False,False,False,False,False,False,False,False
Signature Global City 93,NaN,NaN,NaN,NaN,981.0,1118.0,0.9301,1.06,1235.0,1530.0,...,True,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,NaN,NaN,NaN,NaN,964.0,964.0,NaN,NaN,1127.0,1127.0,...,False,False,False,False,False,False,False,False,False,False
Pyramid Urban Homes 2,335.0,398.0,23.45,0.2786,500.0,625.0,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
Satya The Hermitage,NaN,NaN,NaN,NaN,1450.0,1450.0,NaN,NaN,1991.0,1991.0,...,False,False,False,False,False,False,False,False,False,False


In [47]:
ohe_df.fillna(0,inplace=True)

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# apply scaler - dataframe
ohe_df_normalized = pd.DataFrame( scaler.fit_transform( ohe_df ) , columns=ohe_df.columns, index=ohe_df.index)

In [49]:
ohe_df_normalized.head()

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,...,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,-0.252266,-0.169584,-0.105197,-0.082332,1.223499,1.020101,-0.173712,1.158423,0.553787,0.370864,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
M3M Crown,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.293086,0.472749,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Adani Brahma Samsara Vilasa,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.500583,1.304803,...,-0.289310,-0.063888,2.683282,-0.063888,-0.171139,3.924283,-0.236208,-0.111111,-0.216353,-0.063888
Sobha City,-0.252266,-0.169584,-0.105197,-0.082332,1.240497,1.470610,-0.198425,1.680336,0.405879,0.619632,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Signature Global City 93,-0.252266,-0.169584,-0.105197,-0.082332,0.622383,0.667529,-0.232468,0.295011,-0.100626,-0.070634,...,3.456497,-0.063888,2.683282,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888


In [50]:
# cosine similarity matrix
cosine_sim2 = cosine_similarity(ohe_df_normalized)

In [51]:
cosine_sim2.shape

(246, 246)

In [52]:
def recommend_properties_with_scores(property_name, top_n=247):
    
    sim_scores = list(enumerate(cosine_sim2[ ohe_df_normalized.index.get_loc(property_name) ] ) )
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # indices ,scores  top_n similr properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # geting names of top properties using the indices
    top_properties = ohe_df_normalized.index[top_indices].tolist()
    
    # create df with results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

In [53]:
recommend_properties_with_scores('M3M Golf Hills')

,PropertyName,SimilarityScore
0,AIPL The Peaceful Homes,0.955462
1,Smartworld One DXP,0.954670
2,Unitech Escape,0.953092
3,M3M Capital,0.951156
4,BPTP Terra,0.943128
...,...,...
240,Golden Park,-0.522391
241,Satya Merano Greens,-0.523660
242,ROF Normanton Park,-0.525129
243,BPTP Green Oaks,-0.525286


In [54]:
# next column - locAdv..
df.iloc[2][3]

"{'AIPL Business Club Sector 62': '2.7 Km', 'Heritage Xperiential Learning School': '2 Km', 'CK Birla Hospital': '2.5 Km', 'Paras Trinity Mall Sector 63': '3.5 Km', 'Rapid Metro Station Sector 56': '3.8 Km', 'De Adventure Park': '6.8 Km', 'Golf Course Ext Rd': '99 Meter', 'DoubleTree by Hilton Hotel Gurgaon': '3.6 Km', 'KIIT College of Engineering Sohna Road': '8.4 Km', 'Mehrauli-Gurgaon Road': '11.8 Km', 'Indira Gandhi International Airport': '21.1 Km', 'Nirvana Rd': '160 Meter', 'TERI Golf Course': '8.7 Km'}"

In [55]:
df.iloc[3][3]

"{'The Shikshiyan School': '2.9 KM', 'WTC Plaza': '4 KM', 'Luxus Haritma Resort': '4.2 KM', 'BSF Golf Course': '4.7 KM', 'Dwarka Expressway': '5.1 Km', 'Rions Hospital': '5.6 Km', 'Gurgaon': '5.9 KM', 'The NorthCap University': '9.2 KM', 'Dwarka Sector 21': '9.9 KM', 'Nehru Stadium': '10.7  KM', 'Fun N Food WaterPark': '12.1 KM', 'IGI Airport': '19.3 Km', 'Vasant Kunj': '20.8 KM'}"

In [56]:
df[['PropertyName','LocationAdvantages']]['LocationAdvantages'][0]

"{'Bajghera Road': '800 Meter', 'Palam Vihar Halt': '2.5 KM', 'DPSG Palam Vihar': '3.1 KM', 'Park Hospital': '3.1 KM', 'Gurgaon Railway Station': '4.9 KM', 'The NorthCap University': '5.4 KM', 'Dwarka Expy': '1.2 KM', 'Hyatt Place Gurgaon Udyog Vihar': '7.7 KM', 'Dwarka Sector 21, Metro Station': '7.2 KM', 'Pacific D21 Mall': '7.4 KM', 'Indira Gandhi International Airport': '14.7 KM', 'Hamoni Golf Camp': '6.2 KM', 'Fun N Food Waterpark': '8.8 KM', 'Accenture DDC5': '9 KM'}"

In [57]:
def distance_to_metres(dist_str):
    try :
        if 'Km' in dist_str or 'KM' in dist_str:
            return float(dist_str.split(' ')[0] ) * 1000
        elif 'Meter' in dist_str or 'meter' in dist_str:
            return float(dist_str.split(' ')[0])
        else:
            return None
    
    except:
        return None

In [58]:
import ast
ast.literal_eval(df['LocationAdvantages']).items()

ValueError: malformed node or string: 0      {'Bajghera Road': '800 Meter', 'Palam Vihar Ha...
1      {'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...
2      {'AIPL Business Club Sector 62': '2.7 Km', 'He...
3      {'The Shikshiyan School': '2.9 KM', 'WTC Plaza...
4      {'Pranavananda Int. School': '450 m', 'DLF Sit...
                             ...                        
242    {'Sector 42-43 Metro Station': '1.8 Km', 'Para...
243    {'Aarvy Healthcare Super Speciality': '1.8 KM'...
244    {'Dwarka Expressway': '1.2 Km', 'S N Internati...
245    {'Suncity School': '0.2 Km', 'Gurugram Road': ...
246    {'Sector 84 Road': '600 Meter', 'Delhi Public ...
Name: LocationAdvantages, Length: 246, dtype: object

In [59]:
# extract distance for each location , 
# convert all unique location into a column 
location_keyIndex_valRow= {}
for index , row in df.iterrows(): # fetched single row
    distances = {}
    
    for location, distance in ast.literal_eval(row['LocationAdvantages']).items():
        distances[location] = distance_to_metres(distance)
        
    location_keyIndex_valRow[index] = distances

# convert nested dict out dict key = row , inner dict key col  , val = distance in meters 
location_df = pd.DataFrame.from_dict(location_keyIndex_valRow , orient='index')
location_df.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,...,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Sector 84 Road,Skyview Corporate Park
0,800.0,2500.0,3100.0,3100.0,4900.0,5400.0,1200.0,7700.0,7200.0,7400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,550.0,NaN,NaN,NaN,NaN,6700.0,3800.0,NaN,NaN,7500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,5300.0,NaN,NaN,NaN,2500.0,8800.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,1500.0,NaN,NaN,NaN,6500.0,6700.0,5100.0,NaN,NaN,8200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,5500.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
location_df.columns

Index(['Bajghera Road', 'Palam Vihar Halt', 'DPSG Palam Vihar',
       'Park Hospital', 'Gurgaon Railway Station', 'The NorthCap University',
       'Dwarka Expy', 'Hyatt Place Gurgaon Udyog Vihar',
       'Dwarka Sector 21, Metro Station', 'Pacific D21 Mall',
       ...
       'MCC Cricket Ground Dhankot', 'The Shri Ram School Aravali',
       'Taj City Centre Gurugram', 'Minda Industries  Corporate Office',
       'Rampura Flyover, Naurangpur Rd', 'Manesar toll plaza - Kherki Daula',
       'Imt Manesar, Gurugram', 'Holiday Inn', 'Sector 84 Road',
       'Skyview Corporate Park'],
      dtype='object', length=1070)

In [61]:
location_df.index = df.PropertyName

In [62]:
location_df

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,...,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Sector 84 Road,Skyview Corporate Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,800.0,2500.0,3100.0,3100.0,4900.0,5400.0,1200.0,7700.0,7200.0,7400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
M3M Crown,550.0,NaN,NaN,NaN,NaN,6700.0,3800.0,NaN,NaN,7500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adani Brahma Samsara Vilasa,5300.0,NaN,NaN,NaN,2500.0,8800.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sobha City,1500.0,NaN,NaN,NaN,6500.0,6700.0,5100.0,NaN,NaN,8200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Signature Global City 93,NaN,NaN,NaN,5500.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pyramid Urban Homes 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Satya The Hermitage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# since location_df represent distance between a socity from all landmarks 
# therefore i can not fill zero because it means that a society has all landmarks nearby it as distance is zero 
# so for nan value i m putting a very big value 

location_df.fillna(60000,inplace = True)

In [64]:
from sklearn.preprocessing import StandardScaler

In [65]:
scaler = StandardScaler()

In [66]:
location_df_normalized = pd.DataFrame(scaler.fit_transform(location_df) , columns=location_df.columns, index = location_df.index)

In [67]:
location_df_normalized.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,...,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Sector 84 Road,Skyview Corporate Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,-7.942576,-15.652476,-15.652476,-3.138336,-2.935886,-3.127823,-3.708953,-10.321474,-15.652476,-6.002392,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
M3M Crown,-7.976660,0.063888,0.063888,0.328384,0.369766,-3.045355,-3.532531,0.090354,0.063888,-5.990656,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Adani Brahma Samsara Vilasa,-7.329066,0.063888,0.063888,0.328384,-3.079871,-2.912138,0.280908,0.090354,0.063888,0.171088,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Sobha City,-7.847141,0.063888,0.063888,0.328384,-2.839896,-3.045355,-3.444320,0.090354,0.063888,-5.908499,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Signature Global City 93,0.128494,0.063888,0.063888,-2.992113,0.369766,0.335828,0.280908,0.090354,0.063888,0.171088,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888


In [68]:
cosine_sim3 = cosine_similarity(location_df_normalized)

In [69]:
cosine_sim3.shape

(246, 246)

In [70]:
def recommend_properties_with_scores(property_name, top_n=247):
    
    # this is idea that i would like to introduce flexibility for customer so that they can choose which trait should be given
    # more weightage 
    # here i m just temporarily giving more weigtage to cosine_sim1 recommendor system and other two will get less 
    
    cosine_final_matrix  = 30 * cosine_sim1 + 12 * cosine_sim2 + 9* cosine_sim3
    
    # particular property_name ka index se consine_final_matrix ka row fetch akr lenge
    prop_ind = location_df_normalized.index.get_loc(property_name)
    
    sim_scores = list( enumerate( cosine_final_matrix[ prop_ind ] ) )
    
    sim_scores = sorted(sim_scores,  key=lambda x:x[1] ,reverse=True)# sort properties of similarity scores
    
    top_indices = [i[0] for i in sim_scores[1:top_n+1]]
    top_scores = [i[1] for i in sim_scores[1:top_n+1]]
    
    # Again retrieving property name  
    top_properties = location_df_normalized.index[top_indices].tolist()
    
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScores': top_scores
    })
    
    return recommendations_df

In [71]:
# Test the recommender function using a property name
recommend_properties_with_scores('Ireo Victory Valley')

,PropertyName,SimilarityScores
0,Pioneer Urban Presidia,22.263531
1,Ambience Creacions,20.192702
2,Silverglades The Melia,20.026871
3,DLF The Crest,19.490521
4,AIPL The Peaceful Homes,17.282165
...,...,...
240,Vatika Aspiration,-8.988169
241,Adani Tatva Estates,-9.004478
242,Shree Vardhman City,-9.090036
243,JMS Prime Land,-9.108424


## i need to verify by matching some similar property from csv data and online 
## other  problem is that landmarks sectionn mad 1070 columns which too many landmarks 
## it needs data cleaning intesively to reduce duplicated landmarks 
## for example dwarka expressway  some same loaction dwarka exp and many more 

In [72]:
import joblib

In [73]:
joblib.dump(cosine_sim1, 'cosine_sim_facilities.pkl')
joblib.dump(cosine_sim2, 'cosine_sim_price.pkl')
joblib.dump(cosine_sim3, 'cosine_sim_location.pkl')

['cosine_sim_location.pkl']